In [6]:
!pip install torch torchvision torchaudio



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install torchmetrics


   ---------------------------------------- 0.0/983.0 kB ? eta -:--:--
   --------------------- ------------------ 524.3/983.0 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 983.0/983.0 kB 4.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [10]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
dataloader_train=DataLoader(train_data , batch_size=32,shuffle=True)
dataloader_test=DataLoader(test_data , batch_size=32,shuffle=True)

100.0%
100.0%
100.0%
100.0%


In [11]:
# Start coding here
# Use as many cells as you need
class Net(nn.Module):
   def __init__(self , num_classes):
       super().__init__()
       self.feature_extractor= nn.Sequential(
           nn.Conv2d(1,32 , kernel_size=3 , padding=1),
           nn.ELU(),
           nn.MaxPool2d(kernel_size=2),
           nn.Conv2d(32,64 , kernel_size=3 , padding=1),
           nn.ELU(),
           nn.MaxPool2d(kernel_size=2),
           nn.Flatten()

       )
       self.classifier = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ELU(),
            nn.Linear(128, num_classes)
        )

   def forward(self, x):
        x = self.feature_extractor(x)
        # Pass through classifier
        x = self.classifier(x)
        return x

       

In [ ]:
net= Net(num_classes=10)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001)
for epoch in range(1):
    for images , labels in dataloader_train :
        optimizer.zero_grad()
        predictions=net(images)
        loss=criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        


In [ ]:
# Define the metrics
num_classes=len(train_data.classes)
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
for features, labels in dataloader_test:
    outputs = net(features)
    preds = torch.argmax(outputs, dim=1)
    
    accuracy_metric(preds, labels)
    precision_metric(preds, labels)
    recall_metric(preds, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()

print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)
